# 講談社「Pythonではじめるベイズ機械学習入門」第4章 潜在変数モデル

## 4.1 混合ガウスモデル

In [1]:
using Distributions
using Plots
using Turing
using LinearAlgebra

LoadError: InitError: UndefVarError: Params not defined
during initialization of module Inner

In [ ]:
μ₁ = [
    -1.0
    1.0
]

Σ₁ = [
    0.2 0.0
    0.0 0.2
]

μ₂ = [
    1.0
    -1.0
]

Σ₂ = [
    0.4 0.0
    0.0 0.4
]

p = 0.3

N = 100

X = Array{Float64}(undef, 2, N)
S = Array{Bool}(undef, N)

for i in 1:N
    S[i] = rand(Bernoulli(p))
    (μ, Σ) = S[i] == 1 ? (μ₁, Σ₁) : (μ₂, Σ₂)
    X[:, i] = rand(MvNormal(μ, Σ))
end

scatter(X[1, :], X[2, :], xlabel="x₁", ylabel="x₂", label=:none)

In [ ]:
HalfCauchy(μ, σ) = Truncated(Cauchy(μ, σ), 0.0, Inf)

In [ ]:
@model function gaussian_mixture_model(x)
    D, N = size(x)
    K = 3
    μₖ = Vector(undef, K)
    σₖ = Vector(undef, K)
    zₙ = Vector(undef, N)
    
    for k in 1:K
        μₖ[k] ~ Normal(0.0, 10.0)
        σₖ[k] ~ HalfCauchy(0.0, 3.0)
    end
    
    pi ~ Dirichlet(K, 1.0)
    for n in 1:N
        zₙ[n] ~ Categorical(pi)
        x[:, n] .~ Normal(μₖ[zₙ[n]], σₖ[zₙ[n]])
    end
    
    return zₙ
end

model = gaussian_mixture_model(X)

In [ ]:
chain = sample(model, PG(20), 1000);